<a href="https://colab.research.google.com/github/yingch08/Kaggle-Tytanic/blob/main/0316_Tytanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive

import warnings
warnings.filterwarnings('ignore')

In [4]:
drive.mount('/content/drive')
train = '/content/drive/My Drive/Colab Notebooks/tytanic_train.csv'
train = pd.read_csv(train)
test = '/content/drive/My Drive/Colab Notebooks/tytanic_test.csv'
test = pd.read_csv(test)

Mounted at /content/drive


In [5]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


## 1.1 Exlpore data

In [6]:
# train.describe()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 1.2 Preprocess data: missing values

In [8]:
# Use all data to process
all = pd.concat([train,test], ignore_index=True) # default axis=0
print(all.shape)

(1309, 12)


In [9]:
all.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [10]:
# fill 'Embarked' with mode
all.Embarked.fillna(all.Embarked.mode()[0], inplace=True) # df.mode().iloc[0]) convert df to a series

In [11]:
# raw Fare is sum of fare. Here we calculate single_ticket_fare 'Fare_S'
all['NTickets']=all.groupby('Ticket')['Ticket'].transform('count')
all['Fare_S']=all.Fare/all.NTickets

In [12]:
# fill 'Fare_S' with mean
all.Fare_S.fillna(all.Fare_S.mean(),inplace=True)

In [13]:
# 'Cabin' has too many missing values, so drop this feature
# 'PassengerId', 'Name' are useless, 'Ticket','Fare' are neither anymore.
# Drop all mentioned above
all = all.drop(['PassengerId','Cabin','Name','Ticket','Fare'], axis=1)

In [14]:
# fill 'Age' with mean
all.Age.fillna(all.Age.mean(),inplace=True)

In [15]:
# recheck na
all.isnull().sum()

Survived    418
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Embarked      0
NTickets      0
Fare_S        0
dtype: int64

## 1.3 preprocess data: data type transform

In [16]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    1309 non-null   int64  
 2   Sex       1309 non-null   object 
 3   Age       1309 non-null   float64
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   Embarked  1309 non-null   object 
 7   NTickets  1309 non-null   int64  
 8   Fare_S    1309 non-null   float64
dtypes: float64(3), int64(4), object(2)
memory usage: 92.2+ KB


In [17]:
# 'Sex' mapping F-0, M-1
all['Sex'] = all['Sex'].map({'female':0, 'male':1}).astype(int)

In [18]:
# 'Embarked' get_dummies
dummy_embarked = pd.get_dummies(all['Embarked'],drop_first=True,prefix='Embarked')
all = pd.concat([all,dummy_embarked], axis=1)
all = all.drop(['Embarked'], axis=1)

In [19]:
# recheck 
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    float64
 1   Pclass      1309 non-null   int64  
 2   Sex         1309 non-null   int64  
 3   Age         1309 non-null   float64
 4   SibSp       1309 non-null   int64  
 5   Parch       1309 non-null   int64  
 6   NTickets    1309 non-null   int64  
 7   Fare_S      1309 non-null   float64
 8   Embarked_Q  1309 non-null   uint8  
 9   Embarked_S  1309 non-null   uint8  
dtypes: float64(3), int64(5), uint8(2)
memory usage: 84.5 KB


## 2. split x/y & train/test

In [20]:
x = all.drop(['Survived'],axis=1)
x_train = x.iloc[:891]
x_test = x.iloc[891:]
y_train = all.Survived.iloc[:891]

## 3. Modeling: Tree classifier

In [21]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
Y_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train), 4)
print('Model accuracy (training set) is %.4f' % acc_decision_tree)

Model accuracy (training set) is 0.9820


In [22]:
Submission = pd.DataFrame({'PassengerId': test['PassengerId'],
                           'Survived': Y_pred})

In [23]:
Submission.to_csv('submission.csv', index='False')
!cp submission.csv "drive/My Drive/"

## 3.2 Modeling: random forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf = RandomForestClassifier(n_estimators=100, max_depth=None, oob_score=True)
rf.fit(x_train,y_train)

RandomForestClassifier(oob_score=True)

In [29]:
print('Score (Out of bag data) is %.4f' % rf.oob_score_)

Score (Out of bag data) is 0.7980


In [33]:
y_pred = rf.predict(x_test)
y_pred = pd.DataFrame(y_pred)

In [34]:
y_pred.head()

,0
0,0.0
1,0.0
2,0.0
3,1.0
4,1.0
